<a href="https://colab.research.google.com/github/aglepp25-source/Goose-Egg/blob/main/Goose_Eggs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Load the data
df_2020 = pd.read_csv('/content/drive/MyDrive/Goose Egg/2020plays.zip')

# Inspect relevant columns for runners and scoring
cols_of_interest = ['gid', 'inning', 'top_bot', 'pitcher', 'batter', 'score_v', 'score_h',
                    'br1_pre', 'br2_pre', 'br3_pre', 'outs_pre', 'outs_post',
                    'run1', 'run2', 'run3', 'run_b', 'runs', 'er']
print(df_2020[cols_of_interest].head(10))

# Check distinguishing Starters vs Relievers
# Group by game and find the first pitcher
starters = df_2020.sort_values(['gid', 'inning', 'outs_pre']).groupby('gid')['pitcher'].first()
print("\nSample Starters:")
print(starters.head())

# Check what run1, run2, etc contain
print(df_2020[df_2020['runs'] > 0][['run1', 'run2', 'run3', 'run_b', 'runs']].head())

def evaluate_stint(stint, winner, final_v, final_h, game_df):
    # Determine result

    # 1. Eligibility: Relief Pitcher, Inning >= 7
    if stint['is_starter'] or stint['inning'] < 7:
        return {'pitcher': stint['pitcher'], 'goose': 0, 'broken': 0}

    # 2. Situation
    lead = stint['start_lead']
    runners = stint['start_runners_count']

    # Condition: Lead <= 2 (and >= 0 implied) OR (Lead > 2 and Lead - Runners <= 1)
    # Actually: "Tying run on base or at bat".
    # Tying run score would equal Opponent Score.
    # Current Lead = MyScore - OppScore.
    # Runs needed to tie = Lead.
    # Runners on base = N.
    # Batter = 1.
    # Potential Runs = N + 1.
    # Tying run on base/at bat IF Potential Runs >= Lead.
    # So: (N + 1) >= Lead.

    situation_met = False
    if lead <= 2 and lead >= 0:
        situation_met = True
    elif lead > 0 and (runners + 1) >= lead:
        situation_met = True
    elif lead == 0: # Tied
        situation_met = True

    if not situation_met:
        return {'pitcher': stint['pitcher'], 'goose': 0, 'broken': 0}

    # 3. Goose Egg Criteria
    # A. No runs charged to pitcher in inning
    # B. No inherited runners score while pitcher is in game

    # Runs scored DURING stint
    runs_during_stint = 0
    inherited_scored_during_stint = 0
    earned_runs_charged = 0

    for run_event in stint['runs_allowed_events']:
        runs_during_stint += 1
        if run_event['is_inherited']:
            inherited_scored_during_stint += 1

        if run_event['responsible_pitcher'] == stint['pitcher']:
            # It's his runner.
            if run_event['is_earned_play']:
                earned_runs_charged += 1

    # Check runs charged AFTER stint (but in same inning)
    # Identify runners left on base
    last_play_idx = stint['plays'][-1]
    last_play = game_df.loc[last_play_idx]

    # Runners left
    left_on_base = []
    if pd.notna(last_play['br1_post']): left_on_base.append(last_play['br1_post'])
    if pd.notna(last_play['br2_post']): left_on_base.append(last_play['br2_post'])
    if pd.notna(last_play['br3_post']): left_on_base.append(last_play['br3_post'])

    # Filter for runners responsible by this pitcher
    # (Actually, any runner he leaves is his responsibility if he put them there?
    # No, inherited runners he leaves are still inherited.
    # But wait, did he put them there?
    # We need to check runner_responsibility for these IDs.
    # But I don't have the global map here easily.
    # Solution: We know the 'runs_allowed_events' logic works for future plays IF we process globally.
    # BUT, 'evaluate_stint' is called locally.
    # I need to scan forward in the dataframe for the rest of the inning.

    # Scan forward
    subsequent_plays = game_df.loc[last_play_idx+1:]
    subsequent_plays = subsequent_plays[subsequent_plays['inning'] == stint['inning']]

    runs_charged_later = 0

    # We need to know who put 'left_on_base' runners on base.
    # Since we can't easily access the map, let's look at the runners.
    # If a runner in 'left_on_base' is NOT in 'stint["inherited_runners_ids"]', then the current pitcher put them there.
    # (Assuming no pinch runners changed identity - but IDs usually stable).

    my_left_runners = [r for r in left_on_base if r not in stint['inherited_runners_ids']]

    for _, s_row in subsequent_plays.iterrows():
        # Check if my runners scored
        s_scorers = []
        if pd.notna(s_row['run1']): s_scorers.append(s_row['run1'])
        if pd.notna(s_row['run2']): s_scorers.append(s_row['run2'])
        if pd.notna(s_row['run3']): s_scorers.append(s_row['run3'])

        for s in s_scorers:
            if s in my_left_runners:
                runs_charged_later += 1
                if s_row['er'] > 0:
                    earned_runs_charged += 1

    total_runs_charged = 0
    # From stint
    for re in stint['runs_allowed_events']:
        if re['responsible_pitcher'] == stint['pitcher']:
            total_runs_charged += 1
    total_runs_charged += runs_charged_later

    # Outs Logic
    outs_recorded = stint['outs_end'] - stint['outs_start']
    # Correction: If inning changed, outs_post resets.
    # But we group by inning, so outs_start/end are within one inning.
    # Wait, outs_pre/post are cumulative for inning. Correct.

    goose_egg = False
    if total_runs_charged == 0 and inherited_scored_during_stint == 0:
        if outs_recorded >= 3:
            goose_egg = True
        elif outs_recorded >= 1 and (outs_recorded + stint['start_runners_count']) >= 3:
            goose_egg = True

    # Broken Egg Logic
    broken_egg = False

    # "Could have gotten a goose egg if he'd recorded enough outs"
    # This implies Situation was Met.
    # And failed somewhere?
    # "At least one earned run is charged to the pitcher"
    # "Pitcher does not close out the win"

    # "Could have gotten a GE if he'd recorded enough outs"
    # Does this mean the ONLY reason he failed was outs?
    # Or does it just mean the SITUATION was valid?
    # Usually "Broken Egg" implies he blew it.
    # If he gave up runs, he failed GE.
    # The definition lists 3 AND conditions for Broken Egg:
    # 1. Could have gotten GE (Situation met).
    # 2. ER Charged >= 1.
    # 3. No Close Out.

    # Determine if closed out
    # Pitching team must win.
    is_win = False
    if stint['pitching_team'] == 'HOME' and winner == 'home': is_win = True # Need to map team codes
    # Actually, we have 'winner' as 'home'/'vis'.
    # We know stint['pitching_team'] from 'pitteam'.
    # Compare with 'vis_home' or 'top_bot'.
    # Stint pitteam is just the code (e.g., 'LAN').
    # We need to know if that team won.

    # Check team win
    # Get game info
    row0 = game_df.iloc[0]
    home_team = row0['site'] # Wait, site is stadium.
    # Use 'vis_home' -> 'vis_team', 'home_team'
    # The column names are 'vis_home'?? No. 'vis_team' isn't there.
    # 'vis_home' is likely home team code?
    # Data columns: 'vis_home' (score?), 'batteam', 'pitteam'.
    # Columns: 'gid', 'event', ... 'vis_home' (col 4).
    # Let's check first row content.
    # 'vis_home': 'LOS03' (Site code?).
    # 'batteam': 'SFN', 'pitteam': 'LAN'.
    # 'top_bot': 0. (Top of inning).
    # So LAN is Home, SFN is Vis.

    # Logic:
    # If top_bot=0, pitteam is Home.
    # If top_bot=1, pitteam is Vis.

    pitching_team_won = False
    sample_row_idx = stint['plays'][0]
    s_row = game_df.loc[sample_row_idx]
    if s_row['top_bot'] == 0: # Home Pitching
        pitching_team_won = (winner == 'home')
    else: # Vis Pitching
        pitching_team_won = (winner == 'vis')

    # Close out?
    # Did he finish the game?
    # Check if this stint includes the last play of the game.
    is_last_play_of_game = (game_df.index[-1] in stint['plays'])

    closed_out = (pitching_team_won and is_last_play_of_game)

    if situation_met and (earned_runs_charged >= 1) and (not closed_out):
        broken_egg = True

    return {'pitcher': stint['pitcher'], 'goose': 1 if goose_egg else 0, 'broken': 1 if broken_egg else 0}

def process_data_v2(df_2020):
    results = []

    # Sort by Game, Inning, Top/Bot
    # Assuming file order is correct for intra-inning events
    df_2020 = df_2020.sort_values(['gid', 'inning', 'top_bot', 'outs_pre'])

    # Pre-calculate starters
    starters = df_2020.groupby('gid')['pitcher'].first()

    for gid, game_df in df_2020.groupby('gid'):
        starter_id = starters.loc[gid]

        # Determine Winner
        last_play = game_df.iloc[-1]

        # Calculate final score
        f_v = last_play['score_v']
        f_h = last_play['score_h']
        runs_last = last_play['runs']
        if last_play['top_bot'] == 0:
            f_v += runs_last
        else:
            f_h += runs_last

        winner = 'tie'
        if f_h > f_v: winner = 'home'
        elif f_v > f_h: winner = 'vis'

        runner_resp = {}
        stint = None

        # We need to preserve original order within the game group
        # game_df is a slice, it preserves order from df_2020 (which we sorted by block)
        # But to be safe, we iterate over the sorted df_2020 slice.

        for idx, row in game_df.iterrows():
            pitcher = row['pitcher']
            inning = row['inning']
            top_bot = row['top_bot']

            # 1. Update Runner Map (Post runners)
            post_runners = [row['br1_post'], row['br2_post'], row['br3_post']]
            for r in post_runners:
                if pd.notna(r) and r not in runner_resp:
                    runner_resp[r] = pitcher

            # 2. Check Stint Change
            if stint:
                if stint['pitcher'] != pitcher or stint['inning'] != inning:
                    results.append(evaluate_stint(stint, winner, f_v, f_h, game_df))
                    stint = None

            # 3. Start New Stint
            if stint is None:
                # Situation Context
                if top_bot == 0:
                    lead = row['score_h'] - row['score_v']
                else:
                    lead = row['score_v'] - row['score_h']

                runners_on = []
                if pd.notna(row['br1_pre']): runners_on.append(row['br1_pre'])
                if pd.notna(row['br2_pre']): runners_on.append(row['br2_pre'])
                if pd.notna(row['br3_pre']): runners_on.append(row['br3_pre'])

                stint = {
                    'gid': gid,
                    'pitcher': pitcher,
                    'inning': inning,
                    'is_starter': (pitcher == starter_id),
                    'start_lead': lead,
                    'start_runners_count': len(runners_on),
                    'inherited_runners_ids': runners_on,
                    'outs_start': row['outs_pre'],
                    'outs_end': row['outs_post'],
                    'runs_allowed_events': [],
                    'plays': [idx],
                    'top_bot': top_bot,
                    'pitching_team': row['pitteam'] # Add this for evaluate_stint
                }
            else:
                stint['plays'].append(idx)
                stint['outs_end'] = row['outs_post']

            # 4. Record Runs
            if row['runs'] > 0:
                scorers = [row['run1'], row['run2'], row['run3'], row['run_b']]
                for s in scorers:
                    if pd.notna(s):
                        rp = runner_resp.get(s, starter_id)
                        stint['runs_allowed_events'].append({
                            'runner': s,
                            'responsible_pitcher': rp,
                            'is_inherited': (rp != pitcher),
                            'is_earned_play': (row['er'] > 0)
                        })

        if stint:
            results.append(evaluate_stint(stint, winner, f_v, f_h, game_df))

    return pd.DataFrame(results)

# Run the processing
stats_df = process_data_v2(df_2020)

# Aggregate
final_table = stats_df.groupby('pitcher')[['goose', 'broken']].sum().reset_index()
final_table = final_table[(final_table['goose'] > 0) | (final_table['broken'] > 0)]
final_table = final_table.sort_values(['goose', 'broken'], ascending=[False, True])

print(final_table.head(20))


/tmp/ipython-input-3416362670.py:4: DtypeWarning: Columns (172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv('/content/drive/MyDrive/Goose Egg/2020plays.zip')


            gid  inning  top_bot   pitcher    batter  score_v  score_h  \
0  LAN202007230       1        0  may-d003  yastm001        0        0   
1  LAN202007230       1        0  may-d003  florw001        0        0   
2  LAN202007230       1        0  may-d003  sandp001        0        0   
3  LAN202007230       1        0  may-d003  dicka001        0        0   
4  LAN202007230       1        0  may-d003  pench001        0        0   
5  LAN202007230       1        1  cuetj001  muncm001        0        0   
6  LAN202007230       1        1  cuetj001  bettm001        0        0   
7  LAN202007230       1        1  cuetj001  bellc002        0        0   
8  LAN202007230       2        0  may-d003  mccaj002        0        0   
9  LAN202007230       2        0  may-d003  dubom001        0        0   

    br1_pre   br2_pre br3_pre  outs_pre  outs_post run1 run2 run3 run_b  runs  \
0       NaN       NaN     NaN         0          0  NaN  NaN  NaN   NaN     0   
1  yastm001       NaN  

In [3]:
import pandas as pd

def evaluate_stint(stint, winner, final_v, final_h, game_df):
    # Determine result

    # 1. Eligibility: Relief Pitcher, Inning >= 7
    if stint['is_starter'] or stint['inning'] < 7:
        return {'pitcher': stint['pitcher'], 'goose': 0, 'broken': 0}

    # 2. Situation
    lead = stint['start_lead']
    runners = stint['start_runners_count']

    # Condition: Lead <= 2 (and >= 0 implied) OR (Lead > 2 and Lead - Runners <= 1)
    # Actually: "Tying run on base or at bat".
    # Tying run score would equal Opponent Score.
    # Current Lead = MyScore - OppScore.
    # Runs needed to tie = Lead.
    # Runners on base = N.
    # Batter = 1.
    # Potential Runs = N + 1.
    # Tying run on base/at bat IF Potential Runs >= Lead.
    # So: (N + 1) >= Lead.

    situation_met = False
    if lead <= 2 and lead >= 0:
        situation_met = True
    elif lead > 0 and (runners + 1) >= lead:
        situation_met = True
    elif lead == 0: # Tied
        situation_met = True

    if not situation_met:
        return {'pitcher': stint['pitcher'], 'goose': 0, 'broken': 0}

    # 3. Goose Egg Criteria
    # A. No runs charged to pitcher in inning
    # B. No inherited runners score while pitcher is in game

    # Runs scored DURING stint
    runs_during_stint = 0
    inherited_scored_during_stint = 0
    earned_runs_charged = 0

    for run_event in stint['runs_allowed_events']:
        runs_during_stint += 1
        if run_event['is_inherited']:
            inherited_scored_during_stint += 1

        if run_event['responsible_pitcher'] == stint['pitcher']:
            # It's his runner.
            if run_event['is_earned_play']:
                earned_runs_charged += 1

    # Check runs charged AFTER stint (but in same inning)
    # Identify runners left on base
    last_play_idx = stint['plays'][-1]
    last_play = game_df.loc[last_play_idx]

    # Runners left
    left_on_base = []
    if pd.notna(last_play['br1_post']): left_on_base.append(last_play['br1_post'])
    if pd.notna(last_play['br2_post']): left_on_base.append(last_play['br2_post'])
    if pd.notna(last_play['br3_post']): left_on_base.append(last_play['br3_post'])

    # Filter for runners responsible by this pitcher
    # (Actually, any runner he leaves is his responsibility if he put them there?
    # No, inherited runners he leaves are still inherited.
    # But wait, did he put them there?
    # We need to check runner_responsibility for these IDs.
    # But I don't have the global map here easily.
    # Solution: We know the 'runs_allowed_events' logic works for future plays IF we process globally.
    # BUT, 'evaluate_stint' is called locally.
    # I need to scan forward in the dataframe for the rest of the inning.

    # Scan forward
    subsequent_plays = game_df.loc[last_play_idx+1:]
    subsequent_plays = subsequent_plays[subsequent_plays['inning'] == stint['inning']]

    runs_charged_later = 0

    # We need to know who put 'left_on_base' runners on base.
    # Since we can't easily access the map, let's look at the runners.
    # If a runner in 'left_on_base' is NOT in 'stint["inherited_runners_ids"]', then the current pitcher put them there.
    # (Assuming no pinch runners changed identity - but IDs usually stable).

    my_left_runners = [r for r in left_on_base if r not in stint['inherited_runners_ids']]

    for _, s_row in subsequent_plays.iterrows():
        # Check if my runners scored
        s_scorers = []
        if pd.notna(s_row['run1']): s_scorers.append(s_row['run1'])
        if pd.notna(s_row['run2']): s_scorers.append(s_row['run2'])
        if pd.notna(s_row['run3']): s_scorers.append(s_row['run3'])

        for s in s_scorers:
            if s in my_left_runners:
                runs_charged_later += 1
                if s_row['er'] > 0:
                    earned_runs_charged += 1

    total_runs_charged = 0
    # From stint
    for re in stint['runs_allowed_events']:
        if re['responsible_pitcher'] == stint['pitcher']:
            total_runs_charged += 1
    total_runs_charged += runs_charged_later

    # Outs Logic
    outs_recorded = stint['outs_end'] - stint['outs_start']
    # Correction: If inning changed, outs_post resets.
    # But we group by inning, so outs_start/end are within one inning.
    # Wait, outs_pre/post are cumulative for inning. Correct.

    goose_egg = False
    if total_runs_charged == 0 and inherited_scored_during_stint == 0:
        if outs_recorded >= 3:
            goose_egg = True
        elif outs_recorded >= 1 and (outs_recorded + stint['start_runners_count']) >= 3:
            goose_egg = True

    # Broken Egg Logic
    broken_egg = False

    # "Could have gotten a goose egg if he'd recorded enough outs"
    # This implies Situation was Met.
    # And failed somewhere?
    # "At least one earned run is charged to the pitcher"
    # "Pitcher does not close out the win"

    # "Could have gotten a GE if he'd recorded enough outs"
    # Does this mean the ONLY reason he failed was outs?
    # Or does it just mean the SITUATION was valid?
    # Usually "Broken Egg" implies he blew it.
    # If he gave up runs, he failed GE.
    # The definition lists 3 AND conditions for Broken Egg:
    # 1. Could have gotten GE (Situation met).
    # 2. ER Charged >= 1.
    # 3. No Close Out.

    # Determine if closed out
    # Pitching team must win.
    is_win = False
    if stint['pitching_team'] == 'HOME' and winner == 'home': is_win = True # Need to map team codes
    # Actually, we have 'winner' as 'home'/'vis'.
    # We know stint['pitching_team'] from 'pitteam'.
    # Compare with 'vis_home' or 'top_bot'.
    # Stint pitteam is just the code (e.g., 'LAN').
    # We need to know if that team won.

    # Check team win
    # Get game info
    row0 = game_df.iloc[0]
    home_team = row0['site'] # Wait, site is stadium.
    # Use 'vis_home' -> 'vis_team', 'home_team'
    # The column names are 'vis_home'?? No. 'vis_team' isn't there.
    # 'vis_home' is likely home team code?
    # Data columns: 'gid', 'event', ... 'vis_home' (col 4).
    # Let's check first row content.
    # 'vis_home': 'LOS03' (Site code?).
    # 'batteam': 'SFN', 'pitteam': 'LAN'.
    # 'top_bot': 0. (Top of inning).
    # So LAN is Home, SFN is Vis.

    # Logic:
    # If top_bot=0, pitteam is Home.
    # If top_bot=1, pitteam is Vis.

    pitching_team_won = False
    sample_row_idx = stint['plays'][0]
    s_row = game_df.loc[sample_row_idx]
    if s_row['top_bot'] == 0: # Home Pitching
        pitching_team_won = (winner == 'home')
    else: # Vis Pitching
        pitching_team_won = (winner == 'vis')

    # Close out?
    # Did he finish the game?
    # Check if this stint includes the last play of the game.
    is_last_play_of_game = (game_df.index[-1] in stint['plays'])

    closed_out = (pitching_team_won and is_last_play_of_game)

    if situation_met and (earned_runs_charged >= 1) and (not closed_out):
        broken_egg = True

    return {'pitcher': stint['pitcher'], 'goose': 1 if goose_egg else 0, 'broken': 1 if broken_egg else 0}

def process_data_v2(df_2020):
    results = []

    # Sort by Game, Inning, Top/Bot
    # Assuming file order is correct for intra-inning events
    df_2020 = df_2020.sort_values(['gid', 'inning', 'top_bot', 'outs_pre'])

    # Pre-calculate starters
    starters = df_2020.groupby('gid')['pitcher'].first()

    for gid, game_df in df_2020.groupby('gid'):
        starter_id = starters.loc[gid]

        # Determine Winner
        last_play = game_df.iloc[-1]

        # Calculate final score
        f_v = last_play['score_v']
        f_h = last_play['score_h']
        runs_last = last_play['runs']
        if last_play['top_bot'] == 0:
            f_v += runs_last
        else:
            f_h += runs_last

        winner = 'tie'
        if f_h > f_v: winner = 'home'
        elif f_v > f_h: winner = 'vis'

        runner_resp = {}
        stint = None

        # We need to preserve original order within the game group
        # game_df is a slice, it preserves order from df_2020 (which we sorted by block)
        # But to be safe, we iterate over the sorted df_2020 slice.

        for idx, row in game_df.iterrows():
            pitcher = row['pitcher']
            inning = row['inning']
            top_bot = row['top_bot']

            # 1. Update Runner Map (Post runners)
            post_runners = [row['br1_post'], row['br2_post'], row['br3_post']]
            for r in post_runners:
                if pd.notna(r) and r not in runner_resp:
                    runner_resp[r] = pitcher

            # 2. Check Stint Change
            if stint:
                if stint['pitcher'] != pitcher or stint['inning'] != inning:
                    results.append(evaluate_stint(stint, winner, f_v, f_h, game_df))
                    stint = None

            # 3. Start New Stint
            if stint is None:
                # Situation Context
                if top_bot == 0:
                    lead = row['score_h'] - row['score_v']
                else:
                    lead = row['score_v'] - row['score_h']

                runners_on = []
                if pd.notna(row['br1_pre']): runners_on.append(row['br1_pre'])
                if pd.notna(row['br2_pre']): runners_on.append(row['br2_pre'])
                if pd.notna(row['br3_pre']): runners_on.append(row['br3_pre'])

                stint = {
                    'gid': gid,
                    'pitcher': pitcher,
                    'inning': inning,
                    'is_starter': (pitcher == starter_id),
                    'start_lead': lead,
                    'start_runners_count': len(runners_on),
                    'inherited_runners_ids': runners_on,
                    'outs_start': row['outs_pre'],
                    'outs_end': row['outs_post'],
                    'runs_allowed_events': [],
                    'plays': [idx],
                    'top_bot': top_bot,
                    'pitching_team': row['pitteam'] # Add this for evaluate_stint
                }
            else:
                stint['plays'].append(idx)
                stint['outs_end'] = row['outs_post']

            # 4. Record Runs
            if row['runs'] > 0:
                scorers = [row['run1'], row['run2'], row['run3'], row['run_b']]
                for s in scorers:
                    if pd.notna(s):
                        rp = runner_resp.get(s, starter_id)
                        stint['runs_allowed_events'].append({
                            'runner': s,
                            'responsible_pitcher': rp,
                            'is_inherited': (rp != pitcher),
                            'is_earned_play': (row['er'] > 0)
                        })

        if stint:
            results.append(evaluate_stint(stint, winner, f_v, f_h, game_df))

    return pd.DataFrame(results)

# Load the 2020 data
df_2020 = pd.read_csv('/content/drive/MyDrive/Goose Egg/2020plays.zip')

# Run the processing
stats_df = process_data_v2(df_2020)

# Aggregate
final_table_2020 = stats_df.groupby('pitcher')[['goose', 'broken']].sum().reset_index()
final_table_2020 = final_table_2020[(final_table_2020['goose'] > 0) | (final_table_2020['broken'] > 0)]
final_table_2020 = final_table_2020.sort_values(['goose', 'broken'], ascending=[False, True])

print(final_table_2020.head())

/tmp/ipython-input-1006090982.py:293: DtypeWarning: Columns (172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv('/content/drive/MyDrive/Goose Egg/2020plays.zip')


      pitcher  goose  broken
300  hendl001     21       1
105  castd002     15       0
709  willd004     15       0
22   anden001     15       1
134  coloa001     14       0


In [4]:
import pandas as pd

# The functions evaluate_stint and process_data_v2 are assumed to be defined in the previous cell
# and are not redefined here for brevity, as per coding tip #1.

# List of years to process
years = [2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023, 2024, 2025]

# Dictionary to store final_table for each year
all_final_tables = {'2020': final_table_2020} # Include 2020 from previous step

for year in years:
    print(f"Processing data for year: {year}")
    file_path = f'/content/drive/MyDrive/Goose Egg/{year}plays.zip'

    try:
        # Load the data for the current year, suppress DtypeWarning
        df_year = pd.read_csv(file_path, low_memory=False)

        # Run the processing
        stats_df_year = process_data_v2(df_year)

        # Aggregate
        final_table_year = stats_df_year.groupby('pitcher')[['goose', 'broken']].sum().reset_index()
        final_table_year = final_table_year[(final_table_year['goose'] > 0) | (final_table_year['broken'] > 0)]
        final_table_year = final_table_year.sort_values(['goose', 'broken'], ascending=[False, True])

        # Store in dictionary with year as key
        all_final_tables[str(year)] = final_table_year
        print(f"Finished processing for year: {year}")
    except FileNotFoundError:
        print(f"Warning: Data file not found for year {year} at {file_path}. Skipping this year.")
    except Exception as e:
        print(f"Error processing data for year {year}: {e}. Skipping this year.")

# Concatenate all final_table_YYYY DataFrames into a single complete_goose_egg_dataframe
# Add a 'year' column to each DataFrame before concatenating
for year_key, df in all_final_tables.items():
    df['year'] = int(year_key)

complete_goose_egg_dataframe = pd.concat(all_final_tables.values(), ignore_index=True)

print("\nComplete Goose Egg Dataframe Head:")
print(complete_goose_egg_dataframe.head())
print("\nComplete Goose Egg Dataframe Info:")
complete_goose_egg_dataframe.info()

Processing data for year: 2015
Processing data for year: 2016
Processing data for year: 2017
Processing data for year: 2018
Finished processing for year: 2018
Processing data for year: 2019
Finished processing for year: 2019
Processing data for year: 2021
Finished processing for year: 2021
Processing data for year: 2022
Finished processing for year: 2022
Processing data for year: 2023
Finished processing for year: 2023
Processing data for year: 2024
Finished processing for year: 2024
Processing data for year: 2025
Finished processing for year: 2025

Complete Goose Egg Dataframe Head:
    pitcher  goose  broken  year
0  hendl001     21       1  2020
1  castd002     15       0  2020
2  willd004     15       0  2020
3  anden001     15       1  2020
4  coloa001     14       0  2020

Complete Goose Egg Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3732 entries, 0 to 3731
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  --

In [5]:
import pandas as pd

file_path = '/content/drive/MyDrive/Goose Egg/goose_rawdata.csv'
goose_rawdata_df = pd.read_csv(file_path)

print(goose_rawdata_df.head())
print(goose_rawdata_df.info())

             name  year team league  goose_eggs  broken_eggs  mehs  \
0      Dolf Luque  1921  CIN     NL           3            1     0   
1    Mike Kircher  1921  SLN     NL           0            1     0   
2     Slim Sallee  1921  NY1     NL          13            6     2   
3       Lou North  1921  SLN     NL          10            4     0   
4  Dana Fillingim  1921  BSN     NL           4            1     1   

   league_average_gpct  ppf  replacement_gpct      gwar key_retro  
0             0.752062   94          0.717062  0.068511  luqud101  
1             0.749262   96          0.714262 -0.371416  kircm101  
2             0.746462   98          0.711462 -0.269244  salls101  
3             0.749262   96          0.714262  0.000173  nortl101  
4             0.749262   96          0.714262  0.222919  filld101  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30962 entries, 0 to 30961
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------ 

In [6]:
complete_goose_egg_dataframe = complete_goose_egg_dataframe.rename(columns={'goose': 'goose_eggs', 'broken': 'broken_eggs'})

# Filter goose_rawdata_df for years 2015-2017
historical_years = [2015, 2016, 2017]
historical_goose_egg_data = goose_rawdata_df[goose_rawdata_df['year'].isin(historical_years)].copy()

# Select and rename columns to match complete_goose_egg_dataframe structure
historical_goose_egg_data_cleaned = historical_goose_egg_data[['key_retro', 'goose_eggs', 'broken_eggs', 'year']].copy()
historical_goose_egg_data_cleaned.rename(columns={'key_retro': 'pitcher'}, inplace=True)

# Concatenate with the existing complete_goose_egg_dataframe
complete_goose_egg_dataframe = pd.concat([historical_goose_egg_data_cleaned, complete_goose_egg_dataframe], ignore_index=True)

# Apply the existing filters and sorting
complete_goose_egg_dataframe = complete_goose_egg_dataframe[complete_goose_egg_dataframe['goose_eggs'] > 0]
complete_goose_egg_dataframe = complete_goose_egg_dataframe.sort_values(by=['year', 'goose_eggs'], ascending=[True, False]).reset_index(drop=True)

print(complete_goose_egg_dataframe.head(10))

    pitcher  goose_eggs  broken_eggs  year
0  betad001          44            7  2015
1  melam001          43            2  2015
2  watst001          42            5  2015
3  famij001          41            6  2015
4  smitc004          40            8  2015
5  rondh001          40            4  2015
6  daviw001          39            4  2015
7  kimbc001          39            5  2015
8  tolls002          37            6  2015
9  ziegb001          37            3  2015


In [7]:
complete_goose_egg_dataframe = complete_goose_egg_dataframe.rename(columns={'goose': 'goose_eggs', 'broken': 'broken_eggs'})
print(complete_goose_egg_dataframe.head())

    pitcher  goose_eggs  broken_eggs  year
0  betad001          44            7  2015
1  melam001          43            2  2015
2  watst001          42            5  2015
3  famij001          41            6  2015
4  smitc004          40            8  2015


In [8]:
merged_goose_egg_data = pd.merge(complete_goose_egg_dataframe,
                                   goose_rawdata_df[['key_retro', 'year', 'name', 'team', 'league', 'mehs', 'league_average_gpct', 'ppf', 'replacement_gpct', 'gwar']],
                                   left_on=['pitcher', 'year'],
                                   right_on=['key_retro', 'year'],
                                   how='left')

print(merged_goose_egg_data.head())
print(merged_goose_egg_data.info())

    pitcher  goose_eggs  broken_eggs  year key_retro             name team  \
0  betad001          44            7  2015  betad001  Dellin Betances  NYA   
1  melam001          43            2  2015  melam001    Mark Melancon  PIT   
2  watst001          42            5  2015  watst001      Tony Watson  PIT   
3  famij001          41            6  2015  famij001   Jeurys Familia  NYN   
4  smitc004          40            8  2015  smitc004     Carson Smith  SEA   

  league  mehs  league_average_gpct    ppf  replacement_gpct      gwar  
0     AL   9.0             0.750531  101.0          0.715531  3.904120  
1     NL   6.0             0.773920   97.0          0.738920  5.069271  
2     NL   3.0             0.773920   97.0          0.738920  3.780795  
3     NL   3.0             0.780920   92.0          0.745920  3.089714  
4     AL   8.0             0.760331   94.0          0.725331  2.695739  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4764 entries, 0 to 4763
Data columns (total

In [9]:
goose_egg_data = merged_goose_egg_data[(merged_goose_egg_data['year'] >= 2015) & (merged_goose_egg_data['year'] <= 2025)]
print("Filtered merged_goose_egg_data (first 5 rows):")
print(goose_egg_data.head())
print("\nFiltered merged_goose_egg_data Info:")
goose_egg_data.info()

Filtered merged_goose_egg_data (first 5 rows):
    pitcher  goose_eggs  broken_eggs  year key_retro             name team  \
0  betad001          44            7  2015  betad001  Dellin Betances  NYA   
1  melam001          43            2  2015  melam001    Mark Melancon  PIT   
2  watst001          42            5  2015  watst001      Tony Watson  PIT   
3  famij001          41            6  2015  famij001   Jeurys Familia  NYN   
4  smitc004          40            8  2015  smitc004     Carson Smith  SEA   

  league  mehs  league_average_gpct    ppf  replacement_gpct      gwar  
0     AL   9.0             0.750531  101.0          0.715531  3.904120  
1     NL   6.0             0.773920   97.0          0.738920  5.069271  
2     NL   3.0             0.773920   97.0          0.738920  3.780795  
3     NL   3.0             0.780920   92.0          0.745920  3.089714  
4     AL   8.0             0.760331   94.0          0.725331  2.695739  

Filtered merged_goose_egg_data Info:
<class '

In [10]:
pr_2015 = pd.read_csv('/content/drive/MyDrive/Goose Egg/percentile_rankings_2015.csv')
pr_2016 = pd.read_csv('/content/drive/MyDrive/Goose Egg/percentile_rankings_2016.csv')
pr_2017 = pd.read_csv('/content/drive/MyDrive/Goose Egg/percentile_rankings_2017.csv')
pr_2018 = pd.read_csv('/content/drive/MyDrive/Goose Egg/percentile_rankings_2018.csv')
pr_2019 = pd.read_csv('/content/drive/MyDrive/Goose Egg/percentile_rankings_2019.csv')
pr_2020 = pd.read_csv('/content/drive/MyDrive/Goose Egg/percentile_rankings_2020.csv')
pr_2021 = pd.read_csv('/content/drive/MyDrive/Goose Egg/percentile_rankings_2021.csv')
pr_2022 = pd.read_csv('/content/drive/MyDrive/Goose Egg/percentile_rankings_2022.csv')
pr_2023 = pd.read_csv('/content/drive/MyDrive/Goose Egg/pitcher_percentile_rankings_2023.csv')
pr_2024 = pd.read_csv('/content/drive/MyDrive/Goose Egg/pitcher_percentile_rankings_2024.csv')
pr_2025 = pd.read_csv('/content/drive/MyDrive/Goose Egg/pitcher_percentile_rks_2025.csv')


In [11]:
all_percentile_dfs = []

# List of loaded DataFrames (based on previous cell's output)
df_list = [
    (pr_2015, 2015),
    (pr_2016, 2016),
    (pr_2017, 2017),
    (pr_2018, 2018),
    (pr_2019, 2019),
    (pr_2020, 2020),
    (pr_2021, 2021),
    (pr_2022, 2022),
    (pr_2023, 2023),
    (pr_2024, 2024),
    (pr_2025, 2025)
]

for df, year in df_list:
    temp_df = df[['player_name', 'fb_velocity']].copy() # Select only relevant columns
    temp_df['year'] = year
    all_percentile_dfs.append(temp_df)

percentile_rankings_df = pd.concat(all_percentile_dfs, ignore_index=True)

print("Combined Percentile Rankings DataFrame Head:")
print(percentile_rankings_df.head())
print("\nCombined Percentile Rankings DataFrame Info:")
percentile_rankings_df.info()

Combined Percentile Rankings DataFrame Head:
        player_name  fb_velocity  year
0      Adcock, Nate         73.0  2015
1      Parra, Manny         55.0  2015
2  Cunniff, Brandon         66.0  2015
3    Nolasco, Ricky         24.0  2015
4   Tazawa, Junichi         70.0  2015

Combined Percentile Rankings DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7520 entries, 0 to 7519
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player_name  7520 non-null   object 
 1   fb_velocity  7474 non-null   float64
 2   year         7520 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 176.4+ KB


In [12]:
import pandas as pd
import zipfile
import io

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/Goose Egg/biodata.zip'
csv_file_name = 'biofile0.csv' # The specific file containing player biodata within the zip

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zf:
        # Read the specified CSV file from the zip archive into a BytesIO object
        with zf.open(csv_file_name) as file:
            biodata_df = pd.read_csv(io.BytesIO(file.read()))

    print("Biodata DataFrame Head:")
    print(biodata_df.head())
    print("\nBiodata DataFrame Info:")
    biodata_df.info()
except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_file_path}")
except KeyError:
    print(f"Error: '{csv_file_name}' not found inside {zip_file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Biodata DataFrame Head:
         id lastname usename             fullname   birthdate   birthcity  \
0  aardd001  Aardsma   David  David Allan Aardsma  19811227.0      Denver   
1  aaroh101    Aaron    Hank    Henry Louis Aaron  19340205.0      Mobile   
2  aarot101    Aaron  Tommie     Tommie Lee Aaron  19390805.0      Mobile   
3  aased001     Aase     Don  Donald William Aase  19540908.0      Orange   
4  abada001     Abad    Andy   Fausto Andres Abad  19720825.0  Palm Beach   

   birthstate birthcountry   deathdate deathcity  ...      last_c debut_m  \
0    Colorado          USA         NaN       NaN  ...         NaN     NaN   
1     Alabama          USA  20210122.0   Atlanta  ...         NaN     NaN   
2     Alabama          USA  19840816.0   Atlanta  ...  19840813.0     NaN   
3  California          USA         NaN       NaN  ...         NaN     NaN   
4     Florida          USA         NaN       NaN  ...         NaN     NaN   

  last_m debut_u last_u bats throws height weight 

In [13]:
import pandas as pd
import re

# Define the formatting function to convert 'First Last' to 'Last, First'
def format_name_to_last_first(name_str):
    if pd.isna(name_str) or not isinstance(name_str, str):
        return None

    # Remove text in parentheses and extra spaces
    cleaned_name = re.sub(r'\s*\([^)]*\)', '', name_str).strip()
    parts = cleaned_name.split(' ')

    if len(parts) == 1:
        return cleaned_name
    elif len(parts) == 2: # e.g., 'Aroldis Chapman'
        return f"{parts[1]}, {parts[0]}"
    elif len(parts) >= 3: # e.g., 'Albertin Aroldis Chapman'
        # Assuming the last part is the last name, and the second to last is the primary first name.
        # This is a heuristic, but common for names like 'First Middle Last'.
        return f"{parts[-1]}, {parts[-2]}"
    return cleaned_name # Fallback

# Step 1: Merge goose_egg_data with biodata_df to get the full name for each pitcher.
# The 'pitcher' column in goose_egg_data corresponds to the 'id' in biodata_df.
temp_goose_data_with_bio = pd.merge(
    goose_egg_data,
    biodata_df[['id', 'fullname']],
    left_on='pitcher',
    right_on='id',
    how='left',
    suffixes=('_goose', '_bio')
)

# Step 2: Create a consistently formatted player name ('Last, First') from 'fullname'
temp_goose_data_with_bio['player_name_for_merge'] = temp_goose_data_with_bio['fullname'].apply(format_name_to_last_first)

# Step 3: Now, merge this combined data with percentile_rankings_df.
plotting_df_cleaned = pd.merge(
    temp_goose_data_with_bio,
    percentile_rankings_df,
    left_on=['year', 'player_name_for_merge'],
    right_on=['year', 'player_name'],
    how='left',
    suffixes=('_temp', '_percentile')
)

# Step 4: Consolidate the 'player_name' column.
# Use 'player_name' (from percentile_rankings_df) and fill any NaNs with 'player_name_for_merge' (from biodata)
# The 'player_name' column (from percentile_rankings_df) is NOT renamed to player_name_percentile
# because there is no 'player_name' in temp_goose_data_with_bio to conflict with.
plotting_df_cleaned['player_name_final'] = plotting_df_cleaned['player_name'].fillna(plotting_df_cleaned['player_name_for_merge'])

# Step 5: Drop redundant and temporary columns created during the merge process.
columns_to_drop = [
    'id_bio', 'fullname', 'player_name_for_merge', # These are temporary/intermediate for name creation
    'name', 'key_retro', # These are from the initial merged_goose_egg_data which are not needed for plotting
    'player_name' # This is the 'player_name' column from percentile_rankings_df which is now consolidated into 'player_name_final'
]
# Removed 'player_name_temp' and 'player_name_percentile' as they are not created with these suffixes in this merge scenario.
plotting_df_cleaned.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Rename the final player_name column
plotting_df_cleaned.rename(columns={'player_name_final': 'player_name'}, inplace=True)

# Step 6: Ensure only entries with goose eggs are kept
plotting_df_cleaned = plotting_df_cleaned[plotting_df_cleaned['goose_eggs'] > 0].copy()

# Final checks
print("First 5 rows of plotting_df_cleaned, focusing on key columns:")
print(plotting_df_cleaned[['goose_eggs', 'fb_velocity', 'year', 'player_name']].head())

print("\nInformation about plotting_df_cleaned:")
plotting_df_cleaned.info()

print("\nUnique years in 'year' column:")
print(sorted(plotting_df_cleaned['year'].unique()))

First 5 rows of plotting_df_cleaned, focusing on key columns:
   goose_eggs  fb_velocity  year       player_name
0          44         97.0  2015  Betances, Dellin
1          43          NaN  2015   Melancon, David
2          42          NaN  2015   Watson, Michael
3          41         98.0  2015   Familia, Jeurys
4          40          NaN  2015     Smith, Donald

Information about plotting_df_cleaned:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4764 entries, 0 to 4763
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pitcher              4764 non-null   object 
 1   goose_eggs           4764 non-null   int64  
 2   broken_eggs          4764 non-null   int64  
 3   year                 4764 non-null   int64  
 4   team                 1371 non-null   object 
 5   league               1371 non-null   object 
 6   mehs                 1371 non-null   float64
 7   league_average_gpct  1371 non-

In [14]:
# Merge biodata_df with plotting_df_cleaned to get full names
plotting_df_with_bio = pd.merge(
    plotting_df_cleaned,
    biodata_df[['id', 'fullname']],
    left_on='pitcher', # pitcher column in plotting_df_cleaned contains player ID
    right_on='id',
    how='left'
)

# Define the formatting function to convert 'First Last' to 'Last, First'
def format_name_to_last_first_from_full(name_str):
    if pd.isna(name_str) or not isinstance(name_str, str):
        return None
    parts = name_str.split(' ')
    if len(parts) > 1:
        return f"{parts[-1]}, {' '.join(parts[:-1])}"
    return name_str # Return as is if single word or already formatted

# Create a formatted name from the biodata fullname
plotting_df_with_bio['fullname_formatted'] = plotting_df_with_bio['fullname'].apply(format_name_to_last_first_from_full)

# Fill NaN values in the existing 'player_name' column with the formatted full name from biodata
plotting_df_with_bio['player_name'] = plotting_df_with_bio['player_name'].fillna(
    plotting_df_with_bio['fullname_formatted']
)

# Drop the temporary columns from biodata_df after filling names
plotting_df_cleaned = plotting_df_with_bio.drop(columns=['id', 'fullname', 'fullname_formatted'], errors='ignore')

print("Updated plotting_df_cleaned with filled player names (first 10 rows):")
print(plotting_df_cleaned[['pitcher', 'year', 'player_name']].head(10))
print("\nPlotting_df_cleaned Info after filling names:")
plotting_df_cleaned.info()

Updated plotting_df_cleaned with filled player names (first 10 rows):
    pitcher  year       player_name
0  betad001  2015  Betances, Dellin
1  melam001  2015   Melancon, David
2  watst001  2015   Watson, Michael
3  famij001  2015   Familia, Jeurys
4  smitc004  2015     Smith, Donald
5  rondh001  2015      Rondon, Luis
6  daviw001  2015      Davis, Allen
7  kimbc001  2015  Kimbrel, Michael
8  tolls002  2015    Tolleson, Mark
9  ziegb001  2015  Ziegler, Gregory

Plotting_df_cleaned Info after filling names:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4764 entries, 0 to 4763
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pitcher              4764 non-null   object 
 1   goose_eggs           4764 non-null   int64  
 2   broken_eggs          4764 non-null   int64  
 3   year                 4764 non-null   int64  
 4   team                 1371 non-null   object 
 5   league               1371

In [15]:
entries_per_year = plotting_df_cleaned.groupby('year').size().reset_index(name='number_of_entries')
print(entries_per_year)

    year  number_of_entries
0   2015                464
1   2016                463
2   2017                444
3   2018                461
4   2019                474
5   2020                291
6   2021                428
7   2022                460
8   2023                420
9   2024                413
10  2025                446


In [16]:
df_2025_plot_points = plotting_df_cleaned[
    (plotting_df_cleaned['year'] == 2025) &
    (plotting_df_cleaned['goose_eggs'].notna()) &
    (plotting_df_cleaned['fb_velocity'].notna())
]

num_points_2025 = len(df_2025_plot_points)
print(f"Number of plot points for 2025: {num_points_2025}")

Number of plot points for 2025: 60


In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_for_year(year):
    # Filter data for the current year
    df_year = plotting_df_cleaned[plotting_df_cleaned['year'] == year]

    # Create a new figure and axes for the plot
    fig, ax = plt.subplots(figsize=(10, 7))

    # Generate the scatterplot with a line of best fit
    sns.regplot(data=df_year, x='fb_velocity', y='goose_eggs', scatter_kws={'alpha':0.6}, ax=ax)

    # Set plot title and labels
    ax.set_title(f'Goose Eggs vs. Fastball Velocity ({year})')
    ax.set_xlabel('Fastball Velocity Percentile')
    ax.set_ylabel('Number of Goose Eggs')

    # Set consistent x and y limits
    ax.set_xlim(0, 100) # Percentile values range from 0 to 100
    # Calculate max goose eggs from the entire DataFrame for consistent y-axis
    max_goose_eggs = plotting_df_cleaned['goose_eggs'].max()
    ax.set_ylim(0, max_goose_eggs * 1.1) # Max goose eggs + 10% buffer

    # Add a grid
    ax.grid(True, linestyle='--', alpha=0.7)

    # Display the plot
    plt.show()

print("Function `plot_for_year` defined successfully with line of best fit.")

Function `plot_for_year` defined successfully with line of best fit.


In [18]:
from ipywidgets import interact

# Get the unique years from the DataFrame
years = sorted(plotting_df_cleaned['year'].unique())
min_year = min(years)
max_year = max(years)

# Create the interactive slider
interact(plot_for_year, year=range(min_year, max_year + 1));

interactive(children=(Dropdown(description='year', options=(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 20…

In [19]:
chapman_2025 = plotting_df_cleaned[
    (plotting_df_cleaned['year'] == 2025) &
    (plotting_df_cleaned['player_name'] == 'Chapman, Aroldis')
]

if not chapman_2025.empty:
    print(f"Aroldis Chapman found for 2025:\n{chapman_2025[['player_name', 'goose_eggs', 'fb_velocity']]}")
    if pd.notna(chapman_2025['fb_velocity'].iloc[0]):
        print("Aroldis Chapman should be on the 2025 plot (has fb_velocity data).")
    else:
        print("Aroldis Chapman is in the 2025 data, but will NOT be on the plot (fb_velocity is NaN).")
else:
    print("Aroldis Chapman is NOT found in the 2025 data.")

Aroldis Chapman found for 2025:
           player_name  goose_eggs  fb_velocity
4321  Chapman, Aroldis          40         98.0
Aroldis Chapman should be on the 2025 plot (has fb_velocity data).


In [20]:
plotting_df_cleaned

,pitcher,goose_eggs,broken_eggs,year,team,league,mehs,league_average_gpct,ppf,replacement_gpct,gwar,id_x,fb_velocity,player_name,id_y
0,betad001,44,7,2015,NYA,AL,9.0,0.750531,101.0,0.715531,3.904120,betad001,97.0,"Betances, Dellin",betad001
1,melam001,43,2,2015,PIT,NL,6.0,0.773920,97.0,0.738920,5.069271,melam001,NaN,"Melancon, David",melam001
2,watst001,42,5,2015,PIT,NL,3.0,0.773920,97.0,0.738920,3.780795,watst001,NaN,"Watson, Michael",watst001
3,famij001,41,6,2015,NYN,NL,3.0,0.780920,92.0,0.745920,3.089714,famij001,98.0,"Familia, Jeurys",famij001
4,smitc004,40,8,2015,SEA,AL,8.0,0.760331,94.0,0.725331,2.695739,smitc004,NaN,"Smith, Donald",smitc004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,andei001,1,2,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,andei001,NaN,"Anderson, Theodore",andei001
4760,roycc001,1,2,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,roycc001,NaN,"Roycroft, Michael",roycc001
4761,siske001,1,2,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siske001,6.0,"Sisk, Evan",siske001
4762,chiva001,1,3,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chiva001,88.0,"Chivilli, Angel",chiva001


In [21]:
import plotly.express as px

# Assuming 'plotting_df_cleaned' is your dataframe with columns: 'year', 'fb_velocity', 'goose_eggs', 'player_name'
# animation_frame="year" automatically creates the slider you want!

fig = px.scatter(plotting_df_cleaned,
                 x="fb_velocity",
                 y="goose_eggs",
                 animation_frame="year",  # This creates the player widget/slider
                 hover_name="player_name",       # Shows player name on hover
                 range_x=[0, 100],        # Fix axes so they don't jump around
                 range_y=[0, plotting_df_cleaned['goose_eggs'].max() + 5],
                 title="Fastball Velocity vs. Goose Eggs (By Year)",
                 trendline="ols") # Added regression line
fig.show()


import os
print(f"Your file will be saved here: {os.getcwd()}")
# Save the figure to an HTML file in your current directory
file_name = "my_plot.html"
fig.write_html(file_name)

print(f"File saved to: {os.path.join(os.getcwd(), file_name)}")

Your file will be saved here: /content
File saved to: /content/my_plot.html
